# Wikipedia Text Classifier

## Overview
This Python script utilizes the Wikipedia API and the Natural Language Toolkit (NLTK) library to fetch, preprocess, and classify articles into medical and non-medical categories. The classification is performed using the Naive Bayes classifier from NLTK and preprocess the text before creating features using a Bag of Words approach. In particular the extract_features function utilizes the NLTK library to create a frequency distribution of the preprocessed words for each text.

## Technology Used
- **Programming Language:** Python
- **Libraries:**
  - `wikipediaapi`: Used to interact with the Wikipedia API.
  - `nltk`: The Natural Language Toolkit for natural language processing tasks.
  
## Functions

### `get_wikipedia_content(title)`
- **Description:** Fetches the content of a Wikipedia page with the given title.
- **Parameters:**
  - `title` (str): The title of the Wikipedia page.
- **Returns:** The text content of the Wikipedia page or `None` if the page does not exist.

### `get_category_members(category, num_pages)`
- **Description:** Fetches a specified number of articles from a given Wikipedia category.
- **Parameters:**
  - `category` (str): The name of the Wikipedia category.
  - `num_pages` (int): The number of articles to fetch.
- **Returns:** A list of article contents from the specified category.

### `preprocess_text(text)`
- **Description:** Preprocesses text by tokenizing, converting to lowercase, and removing stop words.
- **Parameters:**
  - `text` (str): The input text to be preprocessed.
- **Returns:** A list of preprocessed words.

### `extract_features(annotated_texts, category)`
- **Description:** Extracts features from annotated texts by creating frequency distributions of preprocessed words.
- **Parameters:**
  - `annotated_texts` (list): A list of annotated texts.
  - `category` (str): The category label for the texts.
- **Returns:** A list of features, where each feature is a tuple containing a frequency distribution and the category label.

### `split_dataset(features)`
- **Description:** Splits the dataset into training and testing sets.
- **Parameters:**
  - `features` (list): The list of features to be split.
- **Returns:** Two lists representing the training set and testing set.

## Main Script (`__main__` block)
1. **Fetch Articles:**
   - Fetches a specified number of medical and non-medical articles from Wikipedia for selected categories.
   - Categories: Medicine, Cardiology, Neurology, Oncology, History, Building, Geography, Mathematics.

2. **Preprocess Texts:**
   - Preprocesses the fetched texts by removing stop words.

3. **Extract Features:**
   - Creates features for medical and non-medical texts using frequency distributions of preprocessed words.

4. **Combine and Shuffle:**
   - Combines medical and non-medical features into a single dataset.
   - Shuffles the dataset to ensure randomness.

5. **Train and Test Classifier:**
   - Splits the dataset into training and testing sets.
   - Trains a Naive Bayes classifier using the training set.
   - Tests the classifier using the testing set and prints the accuracy.

6. **Save Classifier:**
   - Saves the trained classifier to a file (`naive_bayes_classifier.pickle`) for future use.

## Note
- Adjust the `num_medical_articles` and `num_non_medical_articles` variables to control the number of articles fetched for each category.
- The script introduces a delay (`time.sleep(1)`) between API requests to avoid exceeding rate limits.

## Dependencies
- Ensure you have the required libraries installed by running:
  ```bash
  pip install wikipedia-api nltk
  ```

In [ ]:
!pip install wikipedia-api

In [ ]:
import wikipediaapi
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier
import random
import pickle
import time

# Set your own user agent
USER_AGENT = "WikipediaNLTK_Classifier/1.0"

def get_wikipedia_content(title):
    wiki_wiki = wikipediaapi.Wikipedia('en', headers={'User-Agent': USER_AGENT})
    page_py = wiki_wiki.page(title)

    if page_py.exists():
        return page_py.text
    else:
        print(f"Page '{title}' does not exist.")
        return None

def get_category_members(category, num_pages):
    wiki_wiki = wikipediaapi.Wikipedia('en', headers={'User-Agent': 'your-user-agent'})
    category_page = wiki_wiki.page(f"Category:{category}")

    members = list(category_page.categorymembers.values())
    selected_members = members[:num_pages]

    content_list = []
    for member in selected_members:
        title = member.title
        content = get_wikipedia_content(title)
        if content:
            content_list.append(content)

    return content_list

# Function to preprocess text by removing stop words
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
    return filtered_words

# Function to extract features from annotated texts
def extract_features(annotated_texts, category):
    features = []
    for text in annotated_texts:
        preprocessed_text = preprocess_text(text)
        features.append((FreqDist(preprocessed_text), category))
    return features

# Function to split the dataset into training and testing sets
def split_dataset(features):
    split_index = int(0.8 * len(features))
    training_set = features[:split_index]
    testing_set = features[split_index:]
    return training_set, testing_set

if __name__ == "__main__":
    # Fetch n number of medical and non-medical articles in this case 100 for each category
    num_medical_articles = 100
    num_non_medical_articles = 100
    # If the total number is less than that indicated it may happen that the articles in that category are fewer than those requested

    medical_categories = ['Medicine','Cardiology', 'Neurology', 'Oncology']
    non_medical_categories = ['History', 'Building', 'Geography', 'Mathematics']

    medical_texts = []
    num_articles_in_category = 0
    for category in medical_categories:
        medical_texts.extend(get_category_members(category, num_pages=num_medical_articles))
        num_articles_in_category = len(get_category_members(category, num_pages=num_medical_articles))
        print(f"Number of {category} Articles Fetched: {num_articles_in_category}")
        time.sleep(1)  # Introduce a delay between API requests

    non_medical_texts = []
    num_articles_in_category = 0
    for category in non_medical_categories:
        non_medical_texts.extend(get_category_members(category, num_pages=num_non_medical_articles))
        num_articles_in_category = len(get_category_members(category, num_pages=num_medical_articles))
        print(f"Number of {category} Articles Fetched: {num_articles_in_category}")
        time.sleep(1)  # Introduce a delay between API requests

    '''
    ONLY FOR DEBUG

    # Print the content of fetched articles
    print("Medical Articles:")
    for i, text in enumerate(medical_texts):
        print(f"{i + 1}. {text[:50]}...")

    print("\nNon-Medical Articles:")
    for i, text in enumerate(non_medical_texts):
        print(f"{i + 1}. {text[:50]}...")

    '''

    # Extract features from annotated texts
    medical_features = extract_features(medical_texts, 'medical')
    non_medical_features = extract_features(non_medical_texts, 'non-medical')

    # Combine features and shuffle the dataset
    all_features = medical_features + non_medical_features

    # Print debug information
    print(f"\nNumber of medical features: {len(medical_features)}")
    print(f"Number of non-medical features: {len(non_medical_features)}")
    print(f"Total number of features: {len(all_features)}")

    # Check if there are features before shuffling
    if all_features:
        random.shuffle(all_features)

        # Split the dataset into training and testing sets
        training_set, testing_set = split_dataset(all_features)

        # Check if the training set has data
        if training_set:
            # Train the Naive Bayes classifier
            classifier = NaiveBayesClassifier.train(training_set)

            # Test the classifier
            accuracy = nltk.classify.accuracy(classifier, testing_set)
            print(f'Accuracy: {accuracy}')

            # Save the classifier to a file for future use
            with open('naive_bayes_classifier.pickle', 'wb') as file:
                pickle.dump(classifier, file)
        else:
            print("Training set is empty. Check your data.")
    else:
        print("No features extracted. Check your data.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Number of Medicine Articles Fetched: 14
Number of Cardiology Articles Fetched: 100
Number of Neurology Articles Fetched: 100
Number of Oncology Articles Fetched: 100
Number of History Articles Fetched: 35
Number of Building Articles Fetched: 88
Number of Geography Articles Fetched: 60
Number of Mathematics Articles Fetched: 16

Number of medical features: 314
Number of non-medical features: 199
Total number of features: 513
Accuracy: 0.9223300970873787
